In [70]:
import pandas as pd
from bs4 import BeautifulSoup
import glob
from selenium import webdriver
import requests 
import os
import time

In [90]:
driver = webdriver.Chrome('c:/chromedriver.exe')

url ='file:///C:/Users/Playdata/Desktop/shinhancard/shinhancard_20191210_900046432.html'
# url = 'file:///C:/Users/Playdata/Desktop/shinhancard/shinhancard_20201110_900046432.html'
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')
title_tag = soup.select('head > title')

title = title_tag[0].text

In [91]:
title

''

In [ ]:
if title == '체크카드 이메일명세서' :
    

In [71]:
pw = '950730'
driver.find_element_by_id('password').send_keys(pw)
driver.find_element_by_xpath('//*[@id="aform"]/span/input').click()

time.sleep(3)


html = driver.page_source

soup = BeautifulSoup(html,'html.parser')

In [72]:
tables = soup.select('div#email02 > table.tblList.cardUseList > tbody > tr')
# tables = soup.select('div#email02 > table.tblList.cardUseList > tbody > tr > td')
# tables_store = soup.select('div#email02 > table.tblList.cardUseList > tbody > tr > td.textL')
# tables_price = soup.select('div#email02 > table.tblList.cardUseList > tbody > tr > td.textR')
tables_middleSum = soup.select('div#email02 > table.tblList.cardUseList > tbody > tr.middleSum')

tables

[<tr>
 <td>19.10.28</td>
 <td>체크<br/>549</td>
 <td class="textL">서울과학기술대학교소비자생활협동조</td>
 <td class="textR">             1,200</td>
 <td class="textR">                 0</td>
 <td class="textR">                 0</td>
 <td class="textR">             1,200</td>
 <td class="textR">                 0</td>
 <td class="textR">                 0</td>
 <td class="textR"> </td>
 </tr>,
 <tr>
 <td>19.10.28</td>
 <td>체크<br/>549</td>
 <td class="textL">이마트24 노원공릉점</td>
 <td class="textR">             4,000</td>
 <td class="textR">                 0</td>
 <td class="textR">                 0</td>
 <td class="textR">             4,000</td>
 <td class="textR">                 0</td>
 <td class="textR">                 0</td>
 <td class="textR">                 0.20</td>
 </tr>,
 <tr>
 <td>19.10.29</td>
 <td>체크<br/>549</td>
 <td class="textL">맥도날드</td>
 <td class="textR">             5,000</td>
 <td class="textR">                 0</td>
 <td class="textR">                 0</td>
 <td class="textR">   

In [73]:
for n in range(len(tables_middleSum )) :
    tables.remove(tables_middleSum[n])

seperate_tables = []

for n in range(len(tables)) :
    table = str(tables[n].text).split('\n')
    seperate_tables = seperate_tables + table

seperate_tables

['',
 '19.10.28',
 '체크549',
 '서울과학기술대학교소비자생활협동조',
 '             1,200',
 '                 0',
 '                 0',
 '             1,200',
 '                 0',
 '                 0',
 ' ',
 '',
 '',
 '19.10.28',
 '체크549',
 '이마트24 노원공릉점',
 '             4,000',
 '                 0',
 '                 0',
 '             4,000',
 '                 0',
 '                 0',
 '                 0.20',
 '',
 '',
 '19.10.29',
 '체크549',
 '맥도날드',
 '             5,000',
 '                 0',
 '                 0',
 '             5,000',
 '                 0',
 '                 0',
 ' ',
 '',
 '',
 '19.10.29',
 '체크549',
 '다이소(공릉점)',
 '             9,020',
 '                 0',
 '                 0',
 '             9,020',
 '                 0',
 '                 0',
 ' ',
 '',
 '',
 '19.10.30',
 '체크549',
 '이마트24 노원공릉점',
 '             7,300',
 '                 0',
 '                 0',
 '             7,300',
 '                 0',
 '                 0',
 '                 0.20',
 '',

In [74]:
date_list = []

for n in range(1,len(seperate_tables), 12) :
    date = seperate_tables[n]
    date_list.append(date)
    
len(date_list)

94

In [75]:
store_list = []

for n in range(3,len(seperate_tables),12) :
    store = seperate_tables[n].replace(" ", "")
    store_list.append(store)
    
len(store_list)

94

In [76]:
location_list = []

def create_url(search_text):
    headers = {'Authorization': 'KakaoAK 3e8ea332f0cdef6293383ac7bf81a6fc'}

    base = "https://dapi.kakao.com/v2/local/search/keyword.json"
    param_query = "?query=" +search_text
#     print(f'base + param_query: {base + param_query}')
    respond = requests.get(base + param_query,headers=headers)
    return respond.json()

for n in range(len(store_list)) :
# for n in range(10) :
    store_name = store_list[n]
    result = create_url(store_name)
    tmp1 = result['documents']
    try :
        tmp2 = tmp1[0]
        tmp3 = tmp2['address_name']
        location_list.append(tmp3)
    except :
        tmp2 = '해당 주소가 없습니다.'
        location_list.append(tmp2)
#     tmp2 = tmp1[0]
#     tmp3 = tmp2['address_name']
#     location_list.append(tmp3)

In [77]:
len(location_list)

94

In [78]:
price_list = []

for n in range(4,len(seperate_tables),12) :
    price = seperate_tables[n]
    price_list.append(price)

len(price_list)

94

In [79]:
table_data = {'사용날짜':date_list, '사용처':store_list, '사용처주소':location_list,  '사용금액':price_list }

df1 = pd.DataFrame(table_data, columns=['사용날짜', '사용처', '사용처주소', '사용금액'])

df1

,사용날짜,사용처,사용처주소,사용금액
0,19.10.28,서울과학기술대학교소비자생활협동조,해당 주소가 없습니다.,"1,200"
1,19.10.28,이마트24노원공릉점,서울 노원구 공릉동 350-22,"4,000"
2,19.10.29,맥도날드,서울 강남구 역삼동 822-2,"5,000"
3,19.10.29,다이소(공릉점),서울 노원구 공릉동 388-34,"9,020"
4,19.10.30,이마트24노원공릉점,서울 노원구 공릉동 350-22,"7,300"
...,...,...,...,...
89,19.11.18,(주)티머니_지하철,해당 주소가 없습니다.,"1,250"
90,19.11.18,(주)티머니_지하철,해당 주소가 없습니다.,"1,250"
91,19.11.10,ALIEXPRESS,해당 주소가 없습니다.,"50,348"
92,19.11.10,ALIEXPRESS.COM,해당 주소가 없습니다.,"106,299"


In [ ]:
excel_writer = pd.ExcelWriter('c:/myPyCode/data/carduselist.xlsx', 
                              engine='xlsxwriter')
df1.to_excel(excel_writer, index = False, sheet_name='카드사용리스트')
excel_writer.save()